## San Francisco Crime Data Analysis and Modeling (Spark, Python, SQL)

### Use Spark SQL to do big data analysis on SF crime data from 2003 to 2018. ( https://data.sfgov.org/Public-Safety/sf-data/skgt-fej3/data ). 
The first part is OLAP for scrime data analysis.  
The second part is unsupervised learning for spatial data analysis.

Runtime version: 5.4 ML (includes Apache Spark 2.4.0, Scala 2.11)

### PART 0 - Data Preparation

In [4]:
# 5.4 ML runtime verion
from csv import reader
from pyspark.sql import Row, SparkSession
from pyspark.sql.types import *
from pyspark.sql.window import Window 
import pyspark.sql.functions as F
import pandas as pd # 0.23.0
import numpy as np # 1.14.3
import seaborn as sb # 0.8.1
import matplotlib.pyplot as plt # 2.2.2 --> 3.1.0
import matplotlib as mpl
import folium # 0.9.1
import warnings
warnings.filterwarnings("ignore",category=UserWarning) # import 某些library有提示
import urllib.request
import os
os.environ["PYSPARK_PYTHON"] = "python3"

In [5]:
# ------------------------------------------------------------------------------------------------------------------------------------------------
# DATA IMPORT
# ------------------------------------------------------------------------------------------------------------------------------------------------

# 从SF gov 官网读取下载数据
# urllib.request.urlretrieve("https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD", "/tmp/sf_data_2003_2018.csv")
# dbutils.fs.mv("file:/tmp/sf_data_2003_2018.csv", "dbfs:/FileStore/tables/sf_data_2003_2018.csv")
# display(dbutils.fs.ls("dbfs:/laioffer/spark_hw1/data/"))

crime_data_lines = sc.textFile('/FileStore/tables/sf_data_2003_2018.csv') 
df_crimes = crime_data_lines.map(lambda line: [x.strip('"') for x in next(reader([line]))])

# get header --> List<str>
header = df_crimes.first()
print(f'This is what header looks like: \nheader: \n {header}') 

# data without header
crimes = df_crimes.filter(lambda x: x != header) 
crimes.cache()
print('-'*120)

# get the total number of data 
print(f'Now the total crime count is: {crimes.count()}')

# Only use the first thirteen columns, drop the meaningless columns
createRow = Row(*header[0:13]) 
rdd_rows = crimes.map(lambda x : createRow(*x[0:13])) 

df_opt2 = spark.createDataFrame(rdd_rows)
df_opt2.cache()

# ------------------------------------------------------------------------------------------------------------------------------------------------
# DATA PREPROCESSING（duplication, type conversion, missing values)
# ------------------------------------------------------------------------------------------------------------------------------------------------

# 1. Check duplicates
print(f'Check to make sure there are no duplicates: {df_opt2.distinct().count() == df_opt2.count()}.') # both = 2215024 ==> no duplicate rows
# If have，df.drop_duplicates(subset=['colName'])

# 2. Check schema，identify the types need to be converted
print(f'The schema of the dataframe is: ')
df_opt2.printSchema()
# Date --> date --> int (year, month, day)
# Time --> timestamp --> int (hour)
# X, Y --> float
# Resolution --> int ( 0 or 1)

# 3. Check missing values: 
# 1) Null, 
# 2) Empty 
# 3) NaN
missing = df_opt2.select([F.count(F.when( \
                                      (df_opt2[col].isNull()) \
                                      | (df_opt2[col] == '') \
                                      | F.isnan(df_opt2[col]), col)) \
                                      .alias(col) \
                            for col in df_opt2.columns])
with pd.option_context('display.max_rows', 100, 'display.max_columns', 15):
  print(missing.toPandas())
# Only one row has missing value, drop that row
temp1 = df_opt2.filter(df_opt2['PdDistrict'] != '')

# 4. Convert types (according to part2)
temp2 = temp1.withColumn('X', F.col('X').cast(DoubleType())) \
        .withColumn('Y', F.col('Y').cast(DoubleType())) \
        .withColumn('Date_', F.to_date(F.col('Date'), 'MM/dd/yyyy')) \
        .withColumn('Hour', F.hour(F.to_timestamp(F.col('Time'), 'HH:mm'))) \
        .withColumn('Resolution', F.when(F.col('Resolution')!='NONE', 1).otherwise(0)) \
        .drop('IncidntNum','Location')
df = temp2.withColumn('Year', F.year(F.col('Date_'))) \
        .withColumn('Month', F.month(F.col('Date_'))) \
        .withColumn('Day', F.dayofmonth(F.col('Date_'))) 
df.cache()

This is what header looks like: 
header: 
 ['IncidntNum', 'Category', 'Descript', 'DayOfWeek', 'Date', 'Time', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'Location', 'PdId', 'SF Find Neighborhoods', 'Current Police Districts', 'Current Supervisor Districts', 'Analysis Neighborhoods', ':@computed_region_yftq_j783', ':@computed_region_p5aj_wyqh', ':@computed_region_rxqg_mtj9', ':@computed_region_bh8s_q3mv', ':@computed_region_fyvs_ahh9', ':@computed_region_9dfj_4gjx', ':@computed_region_n4xg_c4py', ':@computed_region_4isq_27mq', ':@computed_region_fcz8_est8', ':@computed_region_pigm_ib2e', ':@computed_region_9jxd_iqea', ':@computed_region_6pnf_4xz7', ':@computed_region_6ezc_tdp2', ':@computed_region_h4ep_8xdi', ':@computed_region_nqbw_i6c3', ':@computed_region_2dwj_jsy4']
------------------------------------------------------------------------------------------------------------------------
Now the total crime count is: 2215024
Check to make sure there are no duplicates: True.
The schema of the dataframe is: 
root
-- IncidntNum: string (nullable = true)
-- Category: string (nullable = true)
-- Descript: string (nullable = true)
-- DayOfWeek: string (nullable = true)
-- Date: string (nullable = true)
-- Time: string (nullable = true)
-- PdDistrict: string (nullable = true)
-- Resolution: string (nullable = true)
-- Address: string (nullable = true)
-- X: string (nullable = true)
-- Y: string (nullable = true)
-- Location: string (nullable = true)
-- PdId: string (nullable = true)

 IncidntNum Category Descript DayOfWeek Date Time PdDistrict \
0 0 0 0 0 0 0 1 

 Resolution Address X Y Location PdId 
0 0 0 0 0 0 0 
Out[2]: DataFrame[Category: string, Descript: string, DayOfWeek: string, Date: string, Time: string, PdDistrict: string, Resolution: int, Address: string, X: double, Y: double, PdId: string, Date_: date, Hour: int, Year: int, Month: int, Day: int]

### PART I - OLAP

##### 1. Counts the number of crimes for different category

In [8]:
q1 = df.groupBy('Category').count().sort(F.desc('count')) 
crimes_pd_df = q1.toPandas()

plt.close('all')
fig, ax = plt.subplots(figsize=(14,5))

def format_bar(ax, df, key, value, title_txt, x_txt, y_txt
               , stretch=2.5, width=1.2, align='center', color='SkyBlue', label_rotat=0, label_va='top', label_ha='center', bottom=0):
  labels = df[key]
  data = df[value]
  loc = np.arange(len(labels)) 
  loc_stretch = [stretch*i for i in loc] # Space out bars
  bars = ax.bar(loc_stretch, data, width=width, align=align, color=color, bottom=bottom) #
  ax.spines['top'].set_visible(False)
  ax.spines['right'].set_visible(False)
  ax.set_xticks(loc_stretch)
  ax.set_xticklabels(labels=labels, rotation=label_rotat, fontsize='8', va=label_va, ha=label_ha)
  # horizontalalignment {'center', 'right', 'left'}
  # verticalalignment {'center', 'top', 'bottom', 'baseline', 'center_baseline'}
  ax.get_yaxis().set_major_formatter(
    mpl.ticker.FuncFormatter(lambda x, p : str(x)[:-5]+' K' if x!=0 else '0')) # or format(int(x), ',')
  ax.title.set_text(title_txt)
  ax.set_xlabel(x_txt)
  ax.set_ylabel(y_txt)
  return bars

bars = format_bar(ax, crimes_pd_df, 'Category', 'count', 'Number of Crimes in SF by Category (2003 - 2018.05)', 'Categories of crimes', '# of crimes'
                 , label_rotat=-30, label_ha='left')

# Attach a text label above each bar displaying its height
def bar_autolabel(ax, bars, ha='center', va='bottom', rotation=0):
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., 1.05*height, f'{format(int(height), ",")}', ha=ha, va=va, rotation=rotation)
        
bar_autolabel(ax, bars, ha='left', rotation=30)
        
plt.tight_layout() # Stretch out the plot to fit the figrue
display(fig)

##### 2. Counts the number of crimes for different district, and visualize results

In [10]:
q2_df_1 = df.groupBy('PdDistrict').count().sort(F.desc('count')).withColumnRenamed('count','# of incidents').toPandas()
q2_df_2 = df.groupBy('Category','PdDistrict').count().sort(F.desc('count')).withColumnRenamed('count','# of incidents')

In [11]:
plt.close('all')
fig, ax = plt.subplots(figsize=(14,3))

bars1 = format_bar(ax, q2_df_1, 'PdDistrict', '# of incidents', '# of incidents by districts', 'PdDistrict', '# of incidents')
bar_autolabel(ax, bars1, ha='center')

plt.tight_layout()
display(fig)

In [12]:
display(q2_df_2)

Category,PdDistrict,# of incidents
LARCENY/THEFT,SOUTHERN,113342
LARCENY/THEFT,NORTHERN,80988
LARCENY/THEFT,CENTRAL,71948
OTHER OFFENSES,SOUTHERN,51403
NON-CRIMINAL,SOUTHERN,49821
LARCENY/THEFT,MISSION,48960
OTHER OFFENSES,MISSION,47363
OTHER OFFENSES,BAYVIEW,40805
DRUG/NARCOTIC,TENDERLOIN,38211
OTHER OFFENSES,INGLESIDE,32668


- Analyse the total number of crimes of different districts:
  - Southern, Mission and Northern are the top three districts
- Analyse the percentage of different criminal cases of different districts:
  - If we ignore the percentage of Other Offenses, Larceny is the top one criminal case for most districts. The police may more focus on Larceny;
  - For TenderLoin district, the percentage of Drug/Narcotic is bigger than other criminal cases, even more than Larceny. The police at TenderLoin may pay more attention on drug enforcement.

##### 3. Count the number of crimes each "Sunday" at "SF downtown"

In [15]:
# Take an ellipse area centered on financial district 37.7946° N, 122.3999° W of SF as SF downtown
# East-west axis: 0.005 degree along longitude ~ 426 m, north-south axis: 0.005 degree along latitude ~ 555m

@udf('Boolean')
def within_circle(x, y):
  return (x + 122.3999)**2 + (y - 37.7946)**2 < 0.000025

q3 = df.select('Category', 'DayOfWeek', 'X', 'Y', 'PdId', 'PdDistrict', 'Year', 'Month', 'Day', 'Date_') \
        .where((df.DayOfWeek == "Sunday") & within_circle(df.X, df.Y)) 
display(q3.groupby('Year', 'Month', 'Day', 'Date_').count().sort(['Year', 'Month', 'Day'], ascending=[1, 1, 1]).withColumnRenamed('count', 'Total # of crimes'))

Year,Month,Day,Date_,NumOfCrimes
2003,1,5,2003-01-05,2
2003,1,12,2003-01-12,4
2003,1,19,2003-01-19,1
2003,1,26,2003-01-26,4
2003,2,2,2003-02-02,4
2003,2,9,2003-02-09,1
2003,2,16,2003-02-16,3
2003,2,23,2003-02-23,8
2003,3,2,2003-03-02,2
2003,3,9,2003-03-09,5


The total number of crime on each Sunday seems to be increasing from 2012.

##### 4. Analyse the number of crime in each month of 2015, 2016, 2017, 2018

In [18]:
# pyspark.sql.functions as F

q4 = df.select(df.Year, df.Month, df.PdDistrict, df.Category, df.PdId, df.Resolution) \
        .where(df.Year.isin(*[2015,2016,2017,2018])) \
        .groupby(df.Year, df.Month, df.Category, df.PdDistrict, df.Resolution).count() \
        .sort(df.Year, df.Month, df.Category, df.PdDistrict, df.Resolution)
q4.cache()

Out[9]: DataFrame[Year: int, Month: int, Category: string, PdDistrict: string, Resolution: int, count: bigint]

In [19]:
# Not counting the data from May 2018, because there's only 15 days data for that month

window_q4 = Window.partitionBy('Year', 'Month').orderBy(F.desc('NumOfIndicidnt'))

ans = q4.filter('Year != 2018 OR Month != 5') \
        .groupby(q4.Year, q4.Month, q4.Category).sum('count').withColumnRenamed('sum(count)', 'NumOfIndicidnt')  \
        .withColumn('rnk', F.row_number().over(window_q4)) \
        .filter('rnk <= 5') \
        .sort('Year', 'Month', 'rnk')
display(ans)

Year,Month,Category,NumOfIndicidnt,rnk
2015,1,LARCENY/THEFT,3715,1
2015,1,OTHER OFFENSES,1809,2
2015,1,NON-CRIMINAL,1676,3
2015,1,ASSAULT,1113,4
2015,1,VANDALISM,616,5
2015,2,LARCENY/THEFT,3284,1
2015,2,OTHER OFFENSES,1667,2
2015,2,NON-CRIMINAL,1554,3
2015,2,ASSAULT,1011,4
2015,2,WARRANTS,604,5


- The crime number of larceny is much higher than others. So we can say the total number of crime of SF is dominant by larceny.
- The number of crime seems to be decreasing from 2015 and hit the bottom on 2018, except larceny.
- The number of crime of larceny seems to be increasing from early 2015 and hit the peak on October 2017. Then it decrease fast until April 2018 and hit the bottom.

##### 5. Analysis the number of crime w.r.t the hour in certian day like 2015/12/15, 2016/12/15, 2017/12/15 in order to give the travel suggestion to visit SF.

In [22]:
# filter out the day we are interested in
udf_q5 = udf(lambda x, y, z : x == 12 and y == 15 and (z == 2015 or z == 2016 or z == 2017), BooleanType())

# preprocess
q5 = df.select('Year', 'Month', 'Day', 'Hour', 'Category', 'Pdid') \
        .where(udf_q5(df.Month, df.Day, df.Year)) \
        .groupBy('Year', 'Month', 'Day', 'Hour', 'Category').count()

ans = q5.select('Year', 'Month', 'Day', 'Hour', 'Category', F.col('count').alias('NumOfIncidnt')) \
        .sort('Year', 'Month', 'Day', 'Hour', 'Category')
display(ans)

Year,Month,Day,Hour,Category,NumOfIncidnt
2015,12,15,0,BURGLARY,1
2015,12,15,0,FRAUD,1
2015,12,15,0,LARCENY/THEFT,3
2015,12,15,0,NON-CRIMINAL,1
2015,12,15,0,OTHER OFFENSES,7
2015,12,15,0,ROBBERY,1
2015,12,15,0,SUSPICIOUS OCC,1
2015,12,15,1,ASSAULT,1
2015,12,15,1,DRUNKENNESS,1
2015,12,15,1,OTHER OFFENSES,1


- There's a increasing trend for the number of crime in each hour in Dec 15 from 2015 to 2017. Overall, the number of cirme hits the peak at 12pm and 7pm, and hits the bottom from 1pm to 5pm and 8pm to 9pm.
- According to the trend from 2015 to 2017, the number of crime at 12pm seems to be decreasing. Just for speculation, maybe it is because the police office rearranged the distribution of police due to the high number of crime at 12pm. Until Dec 15 2017, the number of crime from 12pm to 3pm becomes the lesser one. One the contrary, 10am, 7pm and 10pm become the peak.
- For the tracel suggestion to visit SF, during the daytime, I recommend 12pm to 3pm. During the night, 8pm to 9pm is better.

##### 6. (1) Find out the top-3 danger district  
#####    (2) Find out the crime event w.r.t category and time (hour) from the result of step 1  
#####    (3) give advice to distribute the police based on the analysis results

In [25]:
# 1) From part 2 we can tell: The top-3 dangerous districts are Southern, Mission and Northern

# 2) filter out the data we are interested in

# define a window and use the window function to tease out the top 5 frequent category of crimes
window_q6 = Window.partitionBy('Hour').orderBy(F.desc('NumOfIncidnt'))

q6_1 = df.select(df.PdId, df.Hour, df.Category, df.PdDistrict) \
        .filter(df.PdDistrict == 'SOUTHERN') \
        .groupBy('Category', 'Hour').count().withColumnRenamed('count','NumOfIncidnt') \
        .withColumn('rnk', F.row_number().over(window_q6)) \
        .filter('rnk <= 5') \
        .sort('Hour') 

q6_2 = df.select(df.PdId, df.Hour, df.Category, df.PdDistrict) \
        .filter(df.PdDistrict == 'MISSION') \
        .groupBy('Category', 'Hour').count().withColumnRenamed('count','NumOfIncidnt') \
        .withColumn('rnk', F.row_number().over(window_q6)) \
        .filter('rnk <= 5') \
        .sort('Hour') 

q6_3 = df.select(df.PdId, df.Hour, df.Category, df.PdDistrict) \
        .filter(df.PdDistrict == 'NORTHERN') \
        .groupBy('Category', 'Hour').count().withColumnRenamed('count','NumOfIncidnt') \
        .withColumn('rnk', F.row_number().over(window_q6)) \
        .filter('rnk <= 5') \
        .sort('Hour') 

In [26]:
display(q6_1) # Southern

Category,Hour,NumOfIncidnt,rnk
LARCENY/THEFT,0,4163,1
OTHER OFFENSES,0,2947,2
NON-CRIMINAL,0,2414,3
ASSAULT,0,1714,4
FRAUD,0,1174,5
VANDALISM,1,705,5
LARCENY/THEFT,1,2577,1
ASSAULT,1,1506,2
OTHER OFFENSES,1,1415,3
NON-CRIMINAL,1,1243,4


In [27]:
display(q6_2) # Mission

Category,Hour,NumOfIncidnt,rnk
OTHER OFFENSES,0,2754,1
LARCENY/THEFT,0,2554,2
ASSAULT,0,1801,3
NON-CRIMINAL,0,1538,4
VANDALISM,0,868,5
NON-CRIMINAL,1,964,4
LARCENY/THEFT,1,1822,1
ASSAULT,1,1645,2
OTHER OFFENSES,1,1521,3
ROBBERY,1,650,5


In [28]:
display(q6_3) # Northern

Category,Hour,NumOfIncidnt,rnk
LARCENY/THEFT,0,3376,1
OTHER OFFENSES,0,2048,2
NON-CRIMINAL,0,1271,3
ASSAULT,0,1209,4
VANDALISM,0,761,5
NON-CRIMINAL,1,751,4
LARCENY/THEFT,1,2247,1
ASSAULT,1,1188,2
OTHER OFFENSES,1,1024,3
VANDALISM,1,602,5


In [29]:
# define another window to tease out sub-categories
window_q6_2 = Window.partitionBy('Hour', 'Category').orderBy(F.desc('NumOfIncidnt'))

q6_2_2 = df.select(df.PdId, df.Hour, df.Category, df.Descript, df.PdDistrict) \
        .filter(df.PdDistrict == 'MISSION') \
        .groupBy('Category', 'Descript', 'Hour').count().withColumnRenamed('count','NumOfIncidnt') \
        .withColumn('sub_rnk', F.row_number().over(window_q6_2)) \
        .sort('Hour','sub_rnk') 

ans = q6_2.drop('NumOfIncidnt').join(q6_2_2, ['Category', 'Hour'], how = 'left') \
            .filter('rnk <=3 AND sub_rnk <= 3') \
            .select('Category','Descript','Hour','NumOfIncidnt','rnk','sub_rnk') \
            .sort('Hour','rnk','sub_rnk') 
display(ans)

Category,Descript,Hour,NumOfIncidnt,rnk,sub_rnk
OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",0,562,1,1
OTHER OFFENSES,TRAFFIC VIOLATION,0,355,1,2
OTHER OFFENSES,TRAFFIC VIOLATION ARREST,0,249,1,3
LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,0,732,2,1
LARCENY/THEFT,PETTY THEFT OF PROPERTY,0,387,2,2
LARCENY/THEFT,GRAND THEFT OF PROPERTY,0,241,2,3
ASSAULT,BATTERY,0,521,3,1
ASSAULT,THREATS AGAINST LIFE,0,216,3,2
ASSAULT,AGGRAVATED ASSAULT WITH A DEADLY WEAPON,0,203,3,3
LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,1,478,1,1


In [30]:
# to better understanding the result, we need detailed decriptions of category: 'NON-CRIMINAL','OTHER-OFFENSES' and 'LARCENY/THEFT'
dim_category = ans.select('Category', 'Descript') \
                  .filter(F.col('Category').isin(['NON-CRIMINAL', 'OTHER OFFENSES', 'LARCENY/THEFT'])) \
                  .drop_duplicates(['Category', 'Descript']) \
                  .sort('Category', 'Descript')
display(dim_category)

Category,Descript
LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO
LARCENY/THEFT,GRAND THEFT OF PROPERTY
LARCENY/THEFT,PETTY THEFT FROM A BUILDING
LARCENY/THEFT,PETTY THEFT FROM LOCKED AUTO
LARCENY/THEFT,PETTY THEFT OF PROPERTY
LARCENY/THEFT,PETTY THEFT SHOPLIFTING
NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED"
NON-CRIMINAL,FOUND PROPERTY
NON-CRIMINAL,LOST PROPERTY
OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED"


Assumption1: The cases with high number of crime dominant the total number of crime for each district. So only analyse adn visualize the top-5 number of crime cases for each hour.  
Assumption2: Different from other districts, take top-3 number of crime cases of the Mission district.
- From part 2 we can tell: The top-3 dangerous districts are Southern, Mission and Northern.
- For these three districts, the crime concentrates between 12pm to 8pm. The peak is 7pm.
- For Southern and Northern districts, Larceny dominants the number of crime. And the detailed discription of category is Grand theft from locked auto.
- For Mission district, the number of Grand theft from locked auto is much lower than the other two districts. Take the peak as an example, for Southern it's 9k, for Northern it's 6k, but for Mission it's 3k.
- The number of Other Offenses and Non-criminal in Mission district is higher than the other two districts. Based on the detailed discription of category, Other Offenses dominants by Traffic Violation, and Non-criminal dominants by Lost Property.
- Advice:
  - The police in Southern and Northern districts should more focus on the Grand theft from locked auto, and increase the patrol for the high risk area.
  - The police in Mission districts should also pay more attention to Traffic Violation, and improve the traffic flow. For Lost property, strengthening publicity of safe neighborhood may help a little.

##### 7. For different category of crime, calculate the percentage of resolution.

In [33]:
# Only analyse on the data from 2015 to 2018 because of timeliness

q7_1 = df.select(df.Category, df.Resolution, df.Year, df.Month, df.PdId) \
        .filter(df.Year.isin(*[2015, 2016, 2017, 2018])) \
        .filter('Year != 2018 OR Month != 5') \
        .groupBy('Category', 'Resolution').count().withColumnRenamed('count', 'NumOfIndicidnt') \
        .withColumn('% resolved', F.col('NumOfIndicidnt')/F.sum('NumOfIndicidnt').over(Window.partitionBy('Category'))) \
        .filter('Resolution = 0') \
        .drop('Resolution', 'NumOfIndicidnt') \
        .sort(F.col('% resolved').desc()) 
display(q7_1)

Category,% resolved
SUICIDE,0.9561752988047809
LARCENY/THEFT,0.9556050928708194
RUNAWAY,0.9369951534733441
BAD CHECKS,0.9333333333333333
RECOVERED VEHICLE,0.9215189873417722
VEHICLE THEFT,0.9135733556609917
SUSPICIOUS OCC,0.8993207308725185
EXTORTION,0.8959537572254336
MISSING PERSON,0.8878198567041965
VANDALISM,0.8820466221647503


In [34]:
window_q7 = Window.partitionBy('Year', 'Month', 'Category') # 方便计算 resolution percentage

q7_2 = df.select(df.Category, df.Resolution, df.Year, df.Month, df.PdId) \
        .filter(df.Year.isin(*[2015, 2016, 2017, 2018])) \
        .filter('Year != 2018 OR Month != 5') \
        .groupBy('Category', 'Year', 'Month', 'Resolution').count().withColumnRenamed('count', 'NumOfIndicidnt') \
        .withColumn('% resolved', F.col('NumOfIndicidnt')/F.sum('NumOfIndicidnt').over(window_q7)) \
        .filter('Resolution = 0') \
        .drop('Resolution', 'NumOfIndicidnt') \
        .sort(F.col('% resolved').desc()) \
        .filter(F.col('Category').isin(*['LARCENY/THEFT', 'OTHER OFFENSES', 'NON-CRIMINAL', 'ASSAULT', 'VEHICLE THEFT', 'DRUG/NARCOTIC', 'VANDALISM','WARRANTS'])) \
        .sort('Year', 'Month')
display(q7_2)

Category,Year,Month,% resolved
VEHICLE THEFT,2015,1,0.9092409240924092
WARRANTS,2015,1,0.09609120521172639
ASSAULT,2015,1,0.6082659478885895
OTHER OFFENSES,2015,1,0.32780541735765617
DRUG/NARCOTIC,2015,1,0.10268948655256724
LARCENY/THEFT,2015,1,0.9429340511440107
NON-CRIMINAL,2015,1,0.8096658711217184
VANDALISM,2015,1,0.8652597402597403
LARCENY/THEFT,2015,2,0.9482338611449452
DRUG/NARCOTIC,2015,2,0.1111111111111111


- Based on the data from 2015 to 2018, the detection rate of Suicide is the top, and then is Larceny/Theft, Runaway, Bad Checks and Recoverd Vehicle. The detection rate of the top-5 type of cases is over 90%.
- The detection rate of Driving Under The Influence is the lowest one, which is around 5%.
- Tell from the trend of 2015-2018, the detection rate of main type of cases is almost stable, no significant changes.
- Advice for the police:
  - There will be a high risk of Driving Under The Influence. Increasing of the patrol is needed.
  - Letting the citizen involved in the reporting of Driving Under The Influence cases may increase the detection rate of DUI cases.

### PART II - Unsupervised learning for spatial data analysis

##### 1. Apply Spark ML clustering for spatial data analysis

During the data exploratory analysis, I found there are outliers with a latitude 90. It's obvious not SF.

In [39]:
display(df.filter(F.col('Y') == '90'))

Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,PdId,Date_,Hour,Year,Month,Day
LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Friday,12/30/2005,17:00,TENDERLOIN,0,5THSTNORTH ST / OFARRELL ST,-120.5,90.0,5146324506244,2005-12-30,17,2005,12,30
ASSAULT,AGGRAVATED ASSAULT WITH BODILY FORCE,Friday,12/30/2005,00:34,BAYVIEW,1,JAMESLICKFREEWAY HY / SILVER AV,-120.5,90.0,5146011604014,2005-12-30,0,2005,12,30
ASSAULT,INFLICT INJURY ON COHABITEE,Friday,12/30/2005,00:34,BAYVIEW,1,JAMESLICKFREEWAY HY / SILVER AV,-120.5,90.0,5146011615040,2005-12-30,0,2005,12,30
NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED",Thursday,12/29/2005,00:07,TENDERLOIN,1,5THSTNORTH ST / EDDY ST,-120.5,90.0,5145593564020,2005-12-29,0,2005,12,29
OTHER OFFENSES,RESISTING ARREST,Tuesday,12/20/2005,14:57,TENDERLOIN,1,7THSTNORTH ST / MCALLISTER ST,-120.5,90.0,5142583127170,2005-12-20,14,2005,12,20
WARRANTS,WARRANT ARREST,Tuesday,12/20/2005,14:57,TENDERLOIN,1,7THSTNORTH ST / MCALLISTER ST,-120.5,90.0,5142583163010,2005-12-20,14,2005,12,20
OTHER OFFENSES,TRAFFIC VIOLATION,Tuesday,12/20/2005,07:36,RICHMOND,1,GEARY BL / AVENUE OF THE PALMS,-120.5,90.0,5142404365015,2005-12-20,7,2005,12,20
WEAPON LAWS,POSS OF LOADED FIREARM,Monday,12/19/2005,23:08,TENDERLOIN,1,EDDY ST / 5THSTNORTH ST,-120.5,90.0,5142353412100,2005-12-19,23,2005,12,19
OTHER OFFENSES,TRAFFIC VIOLATION,Tuesday,12/06/2005,09:40,RICHMOND,1,GEARY BL / AVENUE OF THE PALMS,-120.5,90.0,5137147265015,2005-12-06,9,2005,12,6
OTHER OFFENSES,TRAFFIC VIOLATION,Wednesday,11/30/2005,11:25,TENDERLOIN,1,5THSTNORTH ST / ELLIS ST,-120.5,90.0,5134899865015,2005-11-30,11,2005,11,30


In [40]:
df.filter(F.col('Y') == '90').count()

Out[42]: 143

In [41]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

# specify assembler
assembler = VectorAssembler(inputCols=["X", "Y"], outputCol="features")
# specify KMeans model
kmeans = KMeans(k=4, seed=1) 

q8 = assembler.transform(df.select(df.X, df.Y, df.Category, df.PdId, df.Address).filter('PdDistrict = "CENTRAL" AND Y != "90"')) 
res = kmeans.fit(q8.select('features')).transform(q8).select('PdId', 'X', 'Y', 'prediction', 'Address', 'Category')

pdf = res.toPandas().sample(frac=0.002)

In [42]:
# 1) create empty map zoomed in on San Francisco
COORD = (37.8006, -122.4099)
my_map = folium.Map(location=COORD
                    , tiles='CartoDB dark_matter' # 'CartoDB dark_matter', 'cartodbpositron'
                    , zoom_start=14
                    , prefer_canvas=True # somehow imporves performance
                    , control_scale=True )
 
# 2) add a marker for every record in the filtered data
def coloring(cluster):
  if cluster == 0: clr = '#ee636b' 
  elif cluster == 1: clr = '#7fc5be'
  elif cluster == 2: clr = '#688eb6'
  else: clr = '#fb9c42'
  return clr

# The 0th element is of type: <class 'int'>, the index of row
# The 1th element is of type: <class 'pandas.core.series.Series'>, the content of the row
for each in pdf.iterrows(): 
    clr = coloring(each[1]['prediction'])
    lat, lon = each[1]['Y'], each[1]['X']
    popup = f"{each[1]['Category']} @ {each[1]['Address']}"
    folium.CircleMarker(location=[lat, lon] 
                  , popup=popup 
                  , radius=3
                  , fill_opacity=1 
                  , fill_color=clr
                 ).add_to(my_map)

# 3) To display interactive map in databricks:
def show_folium_map(m):
  html_str = m._repr_html_() # 转化folium map对象为html code
  width, height = 800, 490
  wrap = f"<iframe srcdoc='{html_str}' width={width} height={height} ></iframe>" # 套上iframe tag
  displayHTML(wrap)

show_folium_map(my_map)

' width=800 height=490 >

In [43]:
%sh
sudo apt-get update && apt-get install -y libspatialindex-dev

Get:1 https://cran.rstudio.com/bin/linux/ubuntu xenial-cran35// InRelease [3,625 B]
Get:2 http://security.ubuntu.com/ubuntu xenial-security InRelease [109 kB]
Hit:3 http://archive.ubuntu.com/ubuntu xenial InRelease
Get:4 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [109 kB]
Get:5 http://archive.ubuntu.com/ubuntu xenial-backports InRelease [107 kB]
Fetched 329 kB in 1s (266 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
libspatialindex-dev is already the newest version (1.8.5-3).
The following package was automatically installed and is no longer required:
 libgnutls-openssl27
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 69 not upgraded.

In [44]:
import geopandas as gpd
from shapely.geometry import Point, Polygon

# 1) Generate shapely.Point from the original data coordinate (X, Y), and cenvert to geopandas dataframe
def generate_geo_df(df):
  geo_points = [Point(xy) for xy in zip(df['X'], df['Y'])]
  return gpd.GeoDataFrame(df, crs={'init' :'epsg:4326'}, geometry=geo_points)

geo_df = generate_geo_df(res.filter('Year IN (2015, 2016, 2017, 2018)').toPandas())

# 2) Set the range of Latitude and Longitud
minlat, maxlat = 37.7846, 37.8076 # Latitude
minlon, maxlon = -122.4259, -122.3899 # Longitud

# 3) Grid the visualized area based on steps, generate shapely.Polygon
def generate_grid(minlat, maxlat, minlon, maxlon, step_size):
  tmp = [] 
  for i in np.arange(minlon, maxlon, step_size):
    for j in np.arange(minlat, maxlat, step_size):
      tmp.append([(i, j), (i+step_size, j), (i+step_size, j+step_size), (i, j+step_size)])
  
  df = pd.Series(tmp).map(Polygon).to_frame().reset_index() 
  df.rename(columns={'index':'Geoid', 0:'Poly'}, inplace=True)
  df['Geoid'] = df['Geoid'].astype(str) 
  gdf = gpd.GeoDataFrame(df, crs={'init' :'epsg:4326'}, geometry=df['Poly'])
  gdf.drop(columns='Poly', inplace=True)
  return gdf

grid = generate_grid(minlat, maxlat, minlon, maxlon, 0.001)

# 4) Merge two dataframe through spatial join from geopandas
temp_df = gpd.sjoin(grid, geo_df, how="inner", op='contains')

# 5) count number of crime 
df_merge = temp_df[['PdId', 'Geoid', 'geometry']].dissolve(by='Geoid', aggfunc='count').rename(columns={'PdId': 'NumCrimes'}).reset_index()

# 6) Draw choropleth map
m = folium.Map(location = (37.8006, -122.4099), tiles='cartodbpositron', zoom_start=14)
# Notice: 'geoid' column that we created earlier needs to be assigned always as the first column
folium.Choropleth(
    geo_data=df_merge,
    name='Crimes in SF Downtown',
    data=df_merge,
    columns=['Geoid', 'NumCrimes'],
    key_on='feature.properties.Geoid',
    fill_color='YlOrRd',
    fill_opacity=0.6,
    line_opacity=0.2,
    line_color='white',
    line_weight=0,
    highlight=False,
    smooth_factor=1.0,
    #threshold_scale=[100, 250, 500, 1000, 2000],
    legend_name= 'Number of Crimes in SF Downtown').add_to(m)

show_folium_map(m)

' width=800 height=490 >

According to the map, Broadway, union square and Pier 39 are the high risk area of crime, which are also the hot places for the traverlers.